In [145]:
import pandas as pd

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

# imported the CSV data
df = pd.read_csv("Filtered_Data.csv", sep = ",")

# Drop the unused column
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis = 1)

df.columns  = [ column_name.split('.')[1] for column_name in df.columns ]

df.head()

,ts,impression_system_date,hour,conversion_status,cc_id_null_ind,ad_id,click_id,cc_id,hour_of_day,raw_publisher_id,...,click_conf_gmt_offset,click_state,click_city,ad_bid,ad_cpa_goal,ad_market_place_id,click_click_status,kmean_category_name,partner_id,version
0,20190106,2019-01-06,1,1,0,138154,646739319951006038,546739320882001606,20,8CUDMGV55,...,-500,ga,atlanta,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
1,20190106,2019-01-06,1,1,0,138154,646739319951006038,546739320882001606,20,8CUDMGV55,...,-500,ga,atlanta,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
2,20190106,2019-01-06,2,0,0,138154,646740715393002311,346740715723006848,20,8CUDMGV55,...,-600,al,birmingham,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
3,20190106,2019-01-06,4,0,0,138154,246748175931002645,446748176329007771,23,8CUDMGV55,...,-500,in,ft wayne,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
4,20190106,2019-01-06,6,0,0,128699,246754980029002839,646754981260002933,22,8CUX8874N,...,-800,ca,mountain view,3.394224,NaN,1027,1,missing,8PRHGG6T9,1


In [146]:
for col in df.columns:
    if(df[col].nunique() == 1):
        df = df.drop(col, axis = 1)
        
# Remove the timestamp data and impression system date data - results not dependent on the time frame
df = df.drop('ts', axis = 1)
df = df.drop('impression_system_date', axis = 1)

# Server hour do not matters
df = df.drop('hour', axis = 1)

# Data have only 3 values of *au* region, and conversion status 0 for all of them
df = df.drop('enriched_country', axis = 1)

# Data is similar to cvr_base_data_table.click_city
df = df.drop('impression_city', axis = 1)

# Ad-Bid is the cost which is irrespecti   ve of conversion from user side
df = df.drop('ad_bid', axis = 1)

# Ad CPA goal do not affects the conversion status
df = df.drop('ad_cpa_goal', axis = 1)

# Click Conf GMT Offset do not affects the conversion status
df = df.drop('click_conf_gmt_offset', axis = 1)

# Date Time do not affects and is too granuler to looku[, thus do not affects the conversion status
df = df.drop('click_user_datetime', axis = 1)

# Publisher Domain do not affects the conversion status
df = df.drop('enriched_publisher_domain', axis = 1)

# Impression ISP Name do not affects the conversion status
df = df.drop('impression_isp_name', axis = 1)

# below data is too granular to predict something
df = df.drop('ad_keyword', axis = 1)
df = df.drop('click_city', axis = 1)
df = df.drop('raw_sub_publisher_id', axis = 1)
df = df.drop('category_name', axis = 1)
df = df.drop('click_id', axis = 1)
df = df.drop('cc_id', axis = 1)
df = df.drop('raw_publisher_id', axis = 1)
df = df.drop('raw_sub_sub_publisher_id', axis = 1)

In [147]:
df.columns

Index(['conversion_status', 'ad_id', 'hour_of_day', 'enriched_derived_device',
       'enriched_derived_os', 'ad_adgroup_id', 'ad_advertiser_id',
       'ad_campaign_id', 'impression_state', 'impression_asn_code',
       'impression_connection_type', 'analyzer_name', 'click_browser',
       'raw_sync_partner_id', 'click_state', 'partner_id'],
      dtype='object')

In [148]:
tots = df.shape[0]
pd.set_option('display.max_rows', 500)

dd = {'conversion_status' : {
    'conversion' : 'sum',
    'click' : 'count'
}}

labelling_columns = ['ad_advertiser_id', 'analyzer_name', 'click_browser', 'enriched_derived_device', 'enriched_derived_os', 'raw_sync_partner_id']

for col in labelling_columns:
    dfg = df.groupby(col).agg(dd).reset_index()
    dfg.columns = dfg.columns.droplevel(0)
    dfg['cvr'] = dfg['conversion'] / dfg['click']
    dfg['click percentage of total data'] = dfg['click'] / tots * 100
    declareMisc = set()
    for index, row in dfg.iterrows():
        if(row['click'] >= 1000):
            continue
        declareMisc.add(row[''])
    for vals in declareMisc:
        df.loc[df[col] == vals, col] = 'misc'
# for col in labelling_columns:
#     dfg = df.groupby(col).agg(dd).reset_index()
#     dfg.columns = dfg.columns.droplevel(0)
#     dfg['cvr'] = dfg['conversion'] / dfg['click']
#     dfg['click percentage of total data'] = dfg['click'] / tots * 100
#     maxPercenatge = dfg['click percentage of total data'].max()

#     markAsMISC = set()
#     for index, row in dfg.iterrows():
#         if(row['click percentage of total data'] / maxPercenatge < 0.013):
#             markAsMISC.add(row[''])

#     # print(markAsMISC)
#     for item in markAsMISC:
#         df = df.replace(item, 'misc')

#     print (df[col].value_counts())

In [151]:
df.shape

print (df.columns)

Index(['conversion_status', 'ad_id', 'hour_of_day', 'enriched_derived_device',
       'enriched_derived_os', 'ad_adgroup_id', 'ad_advertiser_id',
       'ad_campaign_id', 'impression_state', 'impression_asn_code',
       'impression_connection_type', 'analyzer_name', 'click_browser',
       'raw_sync_partner_id', 'click_state', 'partner_id'],
      dtype='object')


In [166]:
for col in labelling_columns:
    dfg = df.groupby(col).agg(dd).reset_index()
    dfg.columns = dfg.columns.droplevel(0)
    dfg['cvr'] = dfg['conversion'] / dfg['click']
    dfg['click percentage of total data'] = dfg['click'] / tots * 100
    
    new_col = dfg['cvr']
    print (new_col.describe(), '\n\n')

count    18.000000
mean      0.084689
std       0.120239
min       0.000000
25%       0.001010
50%       0.030384
75%       0.124726
max       0.423569
Name: cvr, dtype: float64 


count    6.000000
mean     0.063558
std      0.059224
min      0.000114
25%      0.018034
50%      0.063583
75%      0.085351
max      0.158611
Name: cvr, dtype: float64 


count    8.000000
mean     0.045698
std      0.024643
min      0.002725
25%      0.039081
50%      0.049340
75%      0.063215
max      0.073661
Name: cvr, dtype: float64 


count    4.000000
mean     0.043293
std      0.030141
min      0.000000
25%      0.036315
50%      0.051927
75%      0.058906
max      0.069319
Name: cvr, dtype: float64 


count    6.000000
mean     0.053088
std      0.024042
min      0.010101
25%      0.048345
50%      0.055954
75%      0.069055
max      0.077290
Name: cvr, dtype: float64 


count    3.000000
mean     0.065296
std      0.058487
min      0.008107
25%      0.035444
50%      0.062780
75%      0.093890
m